<a href="https://colab.research.google.com/github/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/blob/main/Sentiment_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

In [6]:
fname="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5f1b18584a67905e021e1f495619a124247a5330/Train_all_sentence.fth"


In [7]:
!wget $fname

--2021-10-05 18:42:13--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5f1b18584a67905e021e1f495619a124247a5330/Train_all_sentence.fth
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5f1b18584a67905e021e1f495619a124247a5330/Train_all_sentence.fth [following]
--2021-10-05 18:42:14--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5f1b18584a67905e021e1f495619a124247a5330/Train_all_sentence.fth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2951338 (2.8M) [application/octet-stream]
Saving to: ‘

In [8]:
df=pd.read_feather("Train_all_sentence.fth")

In [9]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text,sentece_code
0,570018445695148033,negative,1.0,Customer Service Issue,0.6560,United,None,tpensari,None,0,@united your customer service is terrible! Sto...,None,2015-02-23 16:32:42 -0800,None,None,your customer service is terrible stood inline...,"[8325, 1798, 6541, 3856, 7342, 7023, 3734, 350..."
1,568830255282040832,neutral,1.0,None,NaN,Southwest,None,leavenodoubt131,None,0,@SouthwestAir I will dm you now,None,2015-02-20 09:51:15 -0800,None,Tijuana,i will do you now,"[3551, 8173, 2175, 8319, 4965]"
2,569296356143050754,negative,1.0,longlines,0.3462,United,None,ScottyNeuman,None,1,@united congratulations united. Fail again. Sk...,"[39.85760174, -104.66669625]",2015-02-21 16:43:22 -0800,"Montana, God's country\n",Pacific Time (US & Canada),congratulations united fail again sky at a gat...,"[1562, 7772, 2657, 153, 6712, 502, 1, 3029, 82..."
3,569936640669581313,positive,1.0,None,NaN,United,None,koploperfan1992,None,0,@united look at this beauty 😉 dc 10 united air...,"[51.322819, 5.3576218]",2015-02-23 11:07:38 -0800,Winterfell,None,look at this beauty do united airlines,"[4323, 502, 7402, 694, 2175, 7772, 197]"
4,569542242983124992,positive,1.0,None,NaN,Delta,None,sandyy889,None,0,@JetBlue Landing! As usual great flight wiyh a...,None,2015-02-22 09:00:26 -0800,None,America/Detroit,landing as usual great flight wiyh a great cre...,"[4092, 457, 7871, 3170, 2829, 8213, 1, 3170, 1..."


In [ ]:

# Importáld a megfelelő rétegeket
from tensorflow.keras.layers import Input,Dense,Embedding,LSTM,TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta,Adam,SGD
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras import backend as K
# Gondolj bele, hogy a hálóban az első réteg egy "beágazás" kell majd legyen
# Ne feledd behozni a funkcionális vagy szekvenciális API-nak megfelelő "fő" osztályt
# Adott esetben az optimalizálót
# Nem külömben a "bakcendet", hogy jó gyakorlat szerint reseteld a gráfot
#tf.reset_default_graph()
K.clear_session
x= Input(shape=(max_input_length,))
embedded_x=Embedding(n_words+1,embedding_size, input_length=max_input_length-1, mask_zero=True)(x)
lstm_output=LSTM(lstm_size,return_sequences=True)(embedded_x)
lstm_output=LSTM(lstm_size,return_sequences=True)(lstm_output)
predictions= Dense(n_words+1, activation="softmax")(lstm_output)
model=Model(inputs=x, outputs=predictions)

